In [22]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt

## We base our first iteration on Unicycle Kinematics.
## https://www.mathworks.com/help/robotics/ug/mobile-robot-kinematics-equations.html
## x = [x,y,heading]^T
## u = [wheel_speed, heading_rate (rad/s)]^T


## Some constants
r = 0.1 # wheel radius

## Define our model
# x_dot = A*x + B*u
A = 0*np.identity(3)
def B(r,x):
    B = np.array([[r*np.cos(x[2]), 0],
                  [r*np.sin(x[2]), 0],
                  [0, 1]])
    return B

In [ ]:
## Tuning parameters
v_k = 0.1*np.array([1.0,1.0,1.0]) # process noise

w_k = 0.0*np.array([1.0,1.0,1.0]) # sensor noise

Q_k = 1*np.array([[0.5,   0,   0], # model noise covariance matrix
                [  0, 0.5,   0],
                [  0,   0, 0.5]])

H_k = np.array([[1.0,  0,   0], # observation matrix (also known as G_k, M_k)
               [  0, 1.0,   0],
               [  0,  0, 1.0]])

R_k = 0.001*np.array([[1.0,   0,    0], # observation/estimate noise covariance matrix
                [  0, 1.0,    0],
                [  0,    0, 1.0]])


In [ ]:
## IEKF function
def IEKF(z_k, x_k_min_1, u_k_min_1, P_k_min_1,iterative,measurement):
    
    # z_k: observation vector
    # x_k_min_1: previous state estimate
    # u_k_min_1: previous input
    # P_k_min_1: previous covariance matrix
    
    # Prediction phase
    x_k = A @ x_k_min_1 + B(r,x_k_min_1) @ (u_k_min_1) + w_k # predicted state estimate (x_frown)
    
    P_k = A @ P_k_min_1 @ A.T + Q_k # updated covariance matrix (pre-correction)
    
    if not measurement:
        return x_k,P_k
    
    
    # Correction phase (EKF)
    S_k = H_k @ P_k @ H_k.T + R_k
    
    y_k_hat = z_k - (H_k @ x_k + v_k) # measurement estimate
    
    K_k = P_k @ H_k.T @ inv(S_k) # near-optimal Kalman gain
    
    x_k_hat = x_k + K_k @ y_k_hat # updated state estimate prediction
    
    P_k = (np.identity(3) - K_k @ H_k) @ P_k # updated covariance matrix (post-correction)
    
    
    # Iterative phase (IEKF)
    if(iterative):
        x_op_k = x_k

        for i in range(10):
            x_k_hat = x_k + K_k @ (z_k - (H_k @ x_op_k + v_k) - H_k @ (x_k - x_op_k)) # updated state estimate prediction
            x_op_k = x_k_hat
    
    # Returning state estimate and covariance
    return x_k_hat, P_k

In [ ]:
def simulate_robot():   
    x_k_min_1 = np.array([0.0,0.0,0.0]) # initial state
    u_k_min_1 = np.array([5,0.1]) # initial input
    P_k_min_1 = 0*np.array([[0.1, 0, 0], # initial covariance
                          [0, 0.1, 0],
                          [0, 0, 0.1]])
    
    x_hat = []
    y_hat = []
    yaw_hat = []
    
    should_iterate = True
    
    #Initial values for simulation
    h = 0.01
    x_sim = np.array([0,0,0]).T #initial simulated states
    
    #Initial input values
    wheelspeed = 5
    headingrate = 0.1
    
    #Bookkeeping lists for plotting
    x_sim_run = []
    x_sim_x = []
    x_sim_y = []
    x_sim_yaw = []
    
    # Time array with step
    t = np.arange(0,25,0.01)
    print(t.size)
    for k in range(t.size):
        measurement = True
        noise_x = np.random.normal(0,v_k[0],1)[0]
        noise_y = np.random.normal(0,v_k[1],1)[0]
        noise_yaw = np.random.normal(0,v_k[2],1)[0]
        u_sim = np.array([wheelspeed, headingrate]).T
        x_sim_dot = A@x_sim + B(r, x_sim)@u_sim
        x_sim = x_sim + x_sim_dot*h
        x_sim_x.append(x_sim[0])
        x_sim_y.append(x_sim[1])
        x_sim_yaw.append(x_sim[2])
        
        if(k%1 == 0):    
            z_k_obs = [x_sim[0]+noise_x,x_sim[1]+noise_y,x_sim[2]+noise_yaw]
            
        else:
            measurement = False
            z_k_obs = [-1,-1,-1]
        
        x_k_hat, P_k = IEKF(z_k_obs, x_k_min_1,u_k_min_1,P_k_min_1,should_iterate,measurement)
         
        # Get ready for the next timestep by updating the variable values
        x_k_min_1 = x_k_hat
        P_k_min_1 = P_k
        
        x_hat.append(x_k_hat[0])
        y_hat.append(x_k_hat[1])
        yaw_hat.append(x_k_hat[2])
    
    plt.plot(x_hat,y_hat, label="trajectory_hat")
    plt.plot(x_sim_x, x_sim_y, label="x and y sim")
    plt.legend()
    plt.show()
    
    plt.plot(t, yaw_hat, label="yaw_hat")
    plt.plot(t, x_sim_yaw, label="yaw_sim")
    plt.legend()
    plt.show()
    

In [ ]:
simulate_robot()